In [1]:
import collections
import sys
import numpy
import numpy as np
#import project_tests as tests
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, SimpleRNN

Using TensorFlow backend.


In [2]:
from keras.layers import Merge
from keras.layers import Lambda
from keras import backend as K
import keras

In [3]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(u"next to the 3 N1 , there are 5 N2 . on the other side of the N3 , there are 7 N2 .")
for token in doc:
    print(token.text, token.pos_, token.head.text, token.head.pos_,
          [child for child in token.children])
noun_key = {}
reverse_noun_key = {}
verb_key = {}
reverse_verb_key = {}
number_key = {}
reverse_number_key = {}

next ADJ are VERB [to]
to ADP next ADJ [N1]
the DET N1 NOUN []
3 NUM N1 NOUN []
N1 NOUN to ADP [the, 3]
, PUNCT are VERB []
there ADV are VERB []
are VERB are VERB [next, ,, there, N2, .]
5 NUM N2 PROPN []
N2 PROPN are VERB [5]
. PUNCT are VERB []
on ADP are VERB [side]
the DET side NOUN []
other ADJ side NOUN []
side NOUN on ADP [the, other, of]
of ADP side NOUN [N3]
the DET N3 NOUN []
N3 NOUN of ADP [the]
, PUNCT are VERB []
there ADV are VERB []
are VERB are VERB [on, ,, there, N2, .]
7 NUM N2 PROPN []
N2 PROPN are VERB [7]
. PUNCT are VERB []


In [5]:
questions = open('abstract_math_questions.txt','r')
labels = open('abstract_math_labels.txt','r')
info_sentences = open('abstract_math_info.txt','r')
fact_sentences = open('abstract_math_facts.txt','r')
standard_relations = open('abstract_math_relations.txt','r')

questions_list = []
labels_list = []
info_list = []
relations_list = []
facts_list = []

for sentence in questions:
    sentence = sentence.lower()
    questions_list.append(sentence)
    
for sentence in labels:
    labels_list.append(sentence)
    
for sentence in info_sentences:
    sentence = sentence.lower()
    info_list.append(sentence)

for sentence in fact_sentences:
    sentence = sentence.lower()
    facts_list.append(sentence)
    
for sentence in standard_relations:
    relations_list.append(sentence)

In [8]:
print(len(questions_list[0]))
print(len(labels_list[0]))
print(len(info_list[0]))
print(len(facts_list[0]))
print(len(relations_list[0]))

24
10
26
5
30


In [9]:
#in this version, we put placeholders for numbers, such as QN1, QN2, etc.. for question numbers

fact_num_dict_list = []
ques_num_dict_list = []
info_num_dict_list = []

#store the numbers in a list of dicts
#for facts list:
for sentence in facts_list:
    fact_num_dict = {}
    indx_num = 1
    for word in sentence:
         if word.isdigit():
            fact_num_dict[word] = "FN%d".format(indx_num)
            word = "FN%d".format(indx_num)
            indx_num += 1
    fact_num_dict_list.append(fact_num_dict)

#repeat for questions
for sentence in questions_list:
    quest_num_dict = {}
    indx_num = 1
    for word in sentence:
         if word.isdigit():
            quest_num_dict[word] = "QN%d".format(indx_num)
            word = "QN%d".format(indx_num)
            indx_num += 1
    ques_num_dict_list.append(quest_num_dict)   

#repeat for information
for sentence in info_list:
    info_num_dict = {}
    indx_num = 1
    for word in sentence:
         if word.isdigit():
            info_num_dict[word] = "IN%d".format(indx_num)
            word = "IN%d".format(indx_num)
            indx_num += 1
    info_num_dict_list.append(info_num_dict)   

#relations do not need since it will already be in encoded format

NameError: name 'nlp' is not defined

In [8]:
list_of_all_variables = []

### Let's make a list of all variables and prepare them for one-hot-coding

# Start by assigning numbers to the standard variables:

list_of_all_variables.append["QN1"]
list_of_all_variables.append["QN2"]
list_of_all_variables.append["QN3"]
list_of_all_variables.append["QN4"]
list_of_all_variables.append["QN5"]
list_of_all_variables.append["QN6"]
list_of_all_variables.append["QN7"]
list_of_all_variables.append["QN8"]
list_of_all_variables.append["FN1"]
list_of_all_variables.append["FN2"]
list_of_all_variables.append["FN3"]
list_of_all_variables.append["FN4"]
list_of_all_variables.append["FN5"]
list_of_all_variables.append["FN6"]
list_of_all_variables.append["FN7"]
list_of_all_variables.append["FN8"]
list_of_all_variables.append["IN1"]
list_of_all_variables.append["IN2"]
list_of_all_variables.append["IN3"]
list_of_all_variables.append["IN4"]
list_of_all_variables.append["IN5"]
list_of_all_variables.append["IN6"]
list_of_all_variables.append["IN7"]
list_of_all_variables.append["IN8"]
list_of_all_variables.append["N1"]
list_of_all_variables.append["N2"]
list_of_all_variables.append["N3"]
list_of_all_variables.append["N4"]
list_of_all_variables.append["N5"]
list_of_all_variables.append["N6"]
list_of_all_variables.append["N7"]
list_of_all_variables.append["N8"]
list_of_all_variables.append["N9"]
list_of_all_variables.append["V1"]
list_of_all_variables.append["V2"]
list_of_all_variables.append["V3"]
list_of_all_variables.append["V4"]
list_of_all_variables.append["V5"]
list_of_all_variables.append["V6"]
list_of_all_variables.append["V7"]
list_of_all_variables.append["V8"]
list_of_all_variables.append["V9"]

for sentence in facts_list:
    for word in sentence:
        if word not in list_of_all_variables:
            list_of_all_variables.append()
    
for sentence in info_list:
    for word in sentence:
        if word not in list_of_all_variables:
            list_of_all_variables.append()

for sentence in questions_list:
    for word in sentence:
        if word not in list_of_all_variables:
            list_of_all_variables.append()

for sentence in relations_list:
    for word in sentence:
        if word not in list_of_all_variables:
            list_of_all_variables.append()      
            
list_of_all_variables.append["PAD"]

In [ ]:
word_to_index = {}
index_to_word = {}

word_index = 0

for word in list_of_all_variables:
    word_to_index[word] = word_index
    index_to_word[word_index] = word
    word_index += 1

In [ ]:
new_facts_list = []
new_questions_list = []
new_info_list = []
new_relations_list = []

pad_up_to = 100

for sentence in facts_list:
    substitute_sentence = []
    for word in sentence:
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
        new_facts_list.append(substitute_sentence)

for sentence in questions_list:
    substitute_sentence = []
    for word in sentence:
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
        new_questions_list.append(substitute_sentence)
        
for sentence in info_list:
    substitute_sentence = []
    for word in sentence:
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
        new_info_list.append(substitute_sentence)

for sentence in relations_list:
    substitute_sentence = []
    for word in sentence:
        substitute_sentence.append(word_to_index[word])
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
        new_relations_list.append(substitute_sentence)

In [9]:
print(len(new_facts_list[0]))
print(len(new_questions_list[0]))
print(len(new_info_list[0]))
print(len(new_relations_list[33]))

100
100
100
300


In [12]:
one_hot_facts = []
one_hot_question = []
one_hot_info = []
one_hot_relations = []

one_hot_length = len(word_to_index)

for sentence in new_facts_list:
    substitute_sentence = []
    for word_ind in sentence:
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_sentence.append[1]
            else:
                substitute_sentence.append[0]
    one_hot_facts.append(substitute_sentence)

for sentence in new_questions_list:
    substitute_sentence = []
    for word_ind in sentence:
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_sentence.append[1]
            else:
                substitute_sentence.append[0]
    one_hot_questions.append(substitute_sentence)

for sentence in new_info_list:
    substitute_sentence = []
    for word_ind in sentence:
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_sentence.append[1]
            else:
                substitute_sentence.append[0]
    one_hot_info.append(substitute_sentence)
    
for sentence in new_relations_list:
    substitute_sentence = []
    for word_ind in sentence:
        for i in range(one_hot_length):
            if i == word_ind:
                substitute_sentence.append[1]
            else:
                substitute_sentence.append[0]
    one_hot_relations.append(substitute_sentence)
    
vectorized_fact_list = np.asarray(one_hot_facts)
vectorized_questions = np.asarray(one_hot_questions) 
vectorized_info = np.asarray(one_hot_info)
vectorized_relations = np.asarray(one_hot_relations)

In [13]:
print(vectorized_fact_list.shape)
print(vectorized_questions.shape)
print(vectorized_info.shape)
print(vectorized_relations.shape)

(34, 100, 6)
(34, 100, 6)
(34, 100, 6)
(34, 300, 6)


In [14]:
label_words_to_index = {}
label_index_to_words = {}
label_index = 0
label_words_to_index["\t"]=label_index
label_index_to_words[label_index]="\t"
label_index += 1
label_words_to_index["<PAD>"]=label_index
label_index_to_words[label_index]="<PAD>"
new_label_list = []
label_pad_up_to = 50
for sentence in labels_list:
    substitute_sentence = []
    sent = sentence.split()
    for char in sent: #char could be a word or group of characters
        if char in label_words_to_index.keys():
            substitute_sentence.append(label_words_to_index[char])
        else:
            label_index += 1
            label_words_to_index[char]=label_index
            label_index_to_words[label_index]=char
            substitute_sentence.append(label_words_to_index[char])
    for i in range(len(substitute_sentence),label_pad_up_to): #pad to regular
        substitute_sentence.append(label_words_to_index["<PAD>"])
        
    new_label_list.append(substitute_sentence)
    
#vectorize labels
vectorized_label_list = []
for sentence in new_label_list:
    sentence_vector = []
    for word in sentence:
        vector = [0]*len(label_words_to_index)
        vector[word] = 1
        sentence_vector.append(vector)
    vectorized_label_list.append(sentence_vector)
    
label_array = np.asarray(vectorized_label_list)

In [15]:
print(label_array.shape)
print(len(label_words_to_index))
print(len(label_index_to_words))

(34, 50, 48)
48
48


In [16]:
def logits_to_text(logits, dictionary):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """    
    return ' '.join([label_index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [17]:
def cascaded_reasoning(info_shape, questions_shape, facts_shape \
                       , relations_shape, labels_index_len):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # TODO: Implement
    learning_rate = 0.01
    hidden_units = 128
    latent_dim = 128 
    # TODO: Build the layers
    
    LSTM_info_inputs = Input(shape=(None, info_shape[2]))
    LSTM_info = LSTM(hidden_units,return_state=True)
    LSTM_info_output, info_h, info_c=LSTM_info(LSTM_info_inputs)
    info_states = [info_h,info_c]
    
    LSTM_questions_inputs = Input(shape=(None, questions_shape[2]))
    LSTM_questions = LSTM(hidden_units,return_state=True)
    LSTM_questions_output, questions_h, questions_c=LSTM_questions(LSTM_questions_inputs)
    questions_states= [questions_h,questions_c]
    
    LSTM_facts_inputs = Input(shape=(None, facts_shape[2]))
    LSTM_facts = LSTM(hidden_units,return_state=True)
    LSTM_facts_output, facts_h, facts_c=LSTM_facts(LSTM_facts_inputs)
    facts_states= [facts_h,facts_c]
    
    LSTM_rela_inputs = Input(shape=(None, relations_shape[2]))
    LSTM_rela = LSTM(hidden_units,return_state=True)
    LSTM_rela_output,rela_h,rela_c=LSTM_rela(LSTM_rela_inputs)
    rela_states= [rela_h,rela_c]
    
    added_h = keras.layers.Add()([info_h,questions_h,facts_h,rela_h])
    added_c = keras.layers.Add()([info_c,questions_c,facts_c,rela_c])
    states = [added_h,added_c]
    
    decoder_inputs = Input(shape=(1, labels_index_len))
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_dense = Dense(labels_index_len, activation='softmax')

    all_outputs = []
    inputs = decoder_inputs
    for _ in range(label_pad_up_to):
        # Run the decoder on one timestep
        outputs, state_h, state_c = decoder_lstm(inputs,
                                             initial_state=states)
        outputs = decoder_dense(outputs)
        # Store the current prediction (we will concatenate all predictions later)
        all_outputs.append(outputs)
        # Reinject the outputs as inputs for the next loop iteration
        # as well as update the states
        inputs = outputs
        states = [state_h, state_c]

    # Concatenate all predictions
    decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)

    # Define and compile model as previously
    model_T = Model([LSTM_info_inputs, LSTM_questions_inputs, LSTM_facts_inputs, \
                     LSTM_rela_inputs, decoder_inputs], decoder_outputs)

    model_T.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model_T

#Example inputs: X1 = "There are four apples at the top of the tree." X2 = "What do you need to get the apples?"
# X3 = "Trees are high."  Label result = "Trees -> High"
#Example inputs: X1 = "There are four apples at the top of the tree." X2 = "What do you need to get the apples?"
# X3 = "Pick apples from tree."  Label result = "Apples -> On tree"
#Example inputs: X1 = "Apples->On tree, Tree->High" X2 = "What do you need to get the apples?"
# X3 = "You can go on tree with a ladder."  Label result = "Ladder"

In [18]:
#tests.test_model_final(model_final)
batch_size = 34

print(len(label_index_to_words))

cascaded_reasoning= cascaded_reasoning(vectorized_info.shape, vectorized_questions.shape, \
                       vectorized_fact_list.shape, vectorized_relations.shape, len(label_index_to_words))

print('cascaded_reasoning loaded')

# Prepare decoder input data that just contains the start character
# Note that we could have made it a constant hard-coded in the model
decoder_input_data = np.zeros((batch_size, 1, len(label_index_to_words)))
decoder_input_data[:, 0, label_words_to_index['\t']] = 1.

cascaded_reasoning.fit([vectorized_info, vectorized_questions, \
                        vectorized_fact_list, vectorized_relations, decoder_input_data], \
                       label_array, batch_size=batch_size, epochs=3, validation_split=0.1)

print("model fit")



48
cascaded_reasoning loaded
Train on 30 samples, validate on 4 samples
Epoch 1/3
30/30 [==============================] - 62s 2s/step - loss: 3.8728 - acc: 0.0033 - val_loss: 3.7941 - val_acc: 0.7800
Epoch 2/3
30/30 [==============================] - 1s 46ms/step - loss: 3.7204 - acc: 0.9080 - val_loss: 3.5629 - val_acc: 0.7800
Epoch 3/3
30/30 [==============================] - 1s 43ms/step - loss: 3.3305 - acc: 0.9080 - val_loss: 2.1994 - val_acc: 0.7800
model fit


In [19]:
hidden_units = 128
latent_dim = 128 
labels_index_len = len(label_index_to_words)
    
LSTM_info_inputs = Input(shape=(None, vectorized_info.shape[2]))
LSTM_info = LSTM(hidden_units,return_state=True)
LSTM_info_output, info_h, info_c=LSTM_info(LSTM_info_inputs)
info_states = [info_h,info_c]
    
LSTM_questions_inputs = Input(shape=(None, vectorized_questions.shape[2]))
LSTM_questions = LSTM(hidden_units,return_state=True)
LSTM_questions_output, questions_h, questions_c=LSTM_questions(LSTM_questions_inputs)
questions_states= [questions_h,questions_c]
    
LSTM_facts_inputs = Input(shape=(None, vectorized_fact_list.shape[2]))
LSTM_facts = LSTM(hidden_units,return_state=True)
LSTM_facts_output, facts_h, facts_c=LSTM_facts(LSTM_facts_inputs)
facts_states= [facts_h,facts_c]
    
LSTM_rela_inputs = Input(shape=(None, vectorized_relations.shape[2]))
LSTM_rela = LSTM(hidden_units,return_state=True)
LSTM_rela_output,rela_h,rela_c=LSTM_rela(LSTM_rela_inputs)
rela_states= [rela_h,rela_c]
    
added_h = keras.layers.Add()([info_h,questions_h,facts_h,rela_h])
added_c = keras.layers.Add()([info_c,questions_c,facts_c,rela_c])
states = [added_h,added_c]

encoder_model = Model([LSTM_info_inputs,LSTM_questions_inputs,LSTM_facts_inputs,LSTM_rela_inputs],states)

fact_model = Model(LSTM_facts_inputs, facts_states)
ques_model = Model(LSTM_questions_inputs, questions_states)
info_model = Model(LSTM_info_inputs, info_states)
rela_model = Model(LSTM_rela_inputs, rela_states)

decoder_inputs = Input(shape=(1, labels_index_len))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(labels_index_len, activation='softmax')

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [20]:
def decode_sequence(info_input, facts_input, ques_input, rela_input,label_words_to_index=label_words_to_index, \
                                        label_index_to_words=label_index_to_words, label_pad_up_to=label_pad_up_to):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict([info_input, facts_input, ques_input, rela_input])

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(label_words_to_index)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, label_words_to_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = label_index_to_words[sampled_token_index]
        decoded_sentence += sampled_char + " "

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > label_pad_up_to):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, len(label_words_to_index)))
        target_seq[0, 0, label_words_to_index['\t']] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [21]:
test_info =  vectorized_info[0,:,:]
test_info = test_info.reshape(1,*test_info.shape) 
test_fact = vectorized_fact_list[0,:,:]
test_fact = test_fact.reshape(1,*test_fact.shape)
test_ques = vectorized_questions[0,:,:]
test_ques = test_ques.reshape(1,*test_ques.shape)
test_rela = vectorized_relations[0,:,:]
test_rela = test_rela.reshape(1,*test_rela.shape)

decoded_sentence = decode_sequence(test_info, test_fact, test_ques, test_rela)

In [22]:
print(decoded_sentence)

EACH EACH EACH 0 0 0 LEN LEN LEN LEN LEN LEN LEN LEN 


In [19]:
# TODO: Train the final model

#Example test:There are 4 doctors working in a clinic. Each doctor has 2 nurses assisting them. There
#are two receptionists, Jay and Molly, working at the reception.
#On Monday, 23 patients made appointments with each doctor. However, 6 of the
#patients did not show up. How many patients visited the clinic on Monday?

nlp = spacy.load('en_core_web_sm')
sentence = "There are 4 doctors working in a clinic. There are 4 doctors working in a clinic. \
          Each doctor has 2 nurses assisting them. There are two receptionists, Jay and Molly, \
          working at the reception. On Monday, 23 patients made appointments with each doctor. \
          However, 6 of the patients did not show up."
question = "How many patients visited the clinic on Monday?"
doc = nlp(sentence)
indx_n = 0
indx_v = 0
indx_num = 0
for token in doc:
    if token.pos_ == "NOUN":
        indx_n += 1
        relations.append("N%d = %s".format(indx_n, token.text))
        noun_key[token.text] = "N%d".format(indx_n)
        reverse_noun_key["N%d".format(indx_n)]=token.text
    if token.pos_ == "VERB":
        indx_v += 1
        relations.append("V%d = %s".format(indx_v, token.text))
        verb_key[token.text] = "V%d".format(indx_v)
        reverse_verb_key["V%d".format(indx_v)]=token.text
    if token.dep_ == "num" or token.dep_ == "num": #this is deprecated
        indx_num += 1
        relations.append("NUM%d = %s".format(indx_num, token.text))
        number_key[token.text] = "NUM%d".format(indx_num)
        reverse_number_key["NUM%d".format(indx_num)]=token.text

#We will not convert the numbers. Instead, we will use vector representations of the numbers. The machine can figure 
#out the relation between the numbers in the sentence. We will represent every word with a 10-digit vector. The 
#first digits will indicate the type of word; for example: representative can be represented by the vector 9000000602,  
#where 9 is for noun. Another example: work = 6000000052, where 6 is for verb. Now, for numbers, there will be zeros in the 
#beginning. For example, 3 will simply be 0000000003, and 24 = 0000000024. We will let the network figure out that these 
#represent actual numbers.   

#create variables to be used in assignments:
var_list = []
for indx_var in range(10):
    var_list[indx_var+1] = "VAR%d".format(indx_var+1) #Add 1 so that VAR list starts with VAR1
    
fact_list[0] = "A physician, medical practitioner, medical doctor, or simply doctor is a professional who practises medicine, \
which is concerned with promoting, maintaining, or restoring health through the study, diagnosis, and treatment \
of disease, injury, and other physical and mental impairments." 

#Now collect info about each of the nouns based on the question:
for noun in noun_list:
    #example: noun being focussed = doctors 
    for fact in fact_list:  
        inputs = [sentence, question, relations, facts]
        #we expect an output that will yield a sort of relation between the words that will be useful to answer the question:
        #these relations will be initially input by humans to train the network on what to look for
        #in this case, the output could be a list of four or five relations, each relation composed of up to four words: 
        #[doctor->practices->medicine, medicine->about->health, 23 patients->made->appointments, 4->doctors]
        #The output from this iteration will be ammended to the relations list and fed into the next iteration.
        #It is possible to have the network find the relations itself by stacking several of these networks. However, the
        #resulting mega-network will be too huge to train. A supercomputer may be necessary.   
        #For now, we will fix the question and info, and iterate over the fact sentences. Labels will correspond to facts. 

NameError: name 'relations' is not defined